# Reddit Comment Database

In [1]:
import os
import glob
import json
import sqlite3
import sqlalchemy
import numpy as np
import pandas as pd
from time import time
from os import listdir
from os.path import isfile, join
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [2]:
dtype_dict = {
    "author": str,
    "author_fullname": str,
    "awarders": str,
    "body": str,
    "id": str,
    "link_id": str,
    "subreddit": str,
    "subreddit_id": str,
    "subreddit_type": str,
}



def count_objects_in_file(filepath):
    """Count how many comments are in a .json file"""
    idx = 0
    with open(filepath) as fp:
        for line in fp:
            idx+=1
    return idx


def extract_subset(filepath, start=0, end=10):
    """Extract a subset of raw comment data directly from .json file"""
    comments = []
    with open(filepath) as fp:
        for idx, line in enumerate(fp):
            if (idx >= start) and (idx < end):
                comment = json.loads(line)
                comments.append(comment)
            elif idx >= end:
                break                
        return comments
        
    
def write_to_database(db_conn, json_fp, chunk_size):
    """Write the contents of temporary .json file to SQLite database"""
    batch_no=1
    for chunk in pd.read_json(json_fp, chunksize=chunk_size, lines=True):
        try: 
            chunk.to_sql('reddit_comments', db_conn, if_exists='append')
        except sqlalchemy.exc.SQLAlchemyError as e: 
            print("\n  {}".format(e.orig))
        batch_no+=1
        
        
def drop_additional_columns(df):
    """Drops specific columns from the dataframe if they exist. This is necessay because some of the 
       comment archives contain additional columns."""
    if "author_cakeday" in df.columns:
        df.drop(columns="author_cakeday", inplace=True)
    if "comment_type" in df.columns:
        df.drop(columns="comment_type", inplace=True)
    if "media_metadata" in df.columns:
        df.drop(columns="media_metadata", inplace=True)
    if "editable" in df.columns:
        df.drop(columns="editable", inplace=True)
    return df
    
        
        
def create_database(database, json_fp, comments_per_chunk, chunk_size, columns_to_drop):
    """
    args:
        database            - sqlite databse object
        json_fp             - filepath to .json raw comments file
        comments_per_chunk  - number of comments to store in temporary .json files 
        chunk_size          - size of chunks for the pd.read_json() function
        columns_to_drop     - columns to drop 
    """
    print("\n######## File: {}".format(json_fp))
    n_comments_total = count_objects_in_file(filepath=json_fp)
    n_chunks = (n_comments_total // comments_per_chunk) + 1
    print("Contains {} comments - dividing into {} chunks".format(n_comments_total, n_chunks))
    
    for idx in range(0, n_chunks):
        start = int(idx * comments_per_chunk)
        end = int(start + comments_per_chunk)
        print("\r   File chunk {}: Extracting and writing comments {} - {}".format(idx+1, start, end), end='')
        comments = extract_subset(filepath=json_fp, start=start, end=end)
        df = pd.DataFrame(comments)
        df = df.drop(columns=columns_to_drop)
        df = drop_additional_columns(df=df)
        df = df.astype(dtype_dict)
        # Load the chunk into temporary .json file
        df.to_json("data/db_chunk.json", orient='records', lines=True)
        write_to_database(
            db_conn=database, 
            json_fp="data/db_chunk.json", 
            chunk_size=chunk_size
        )
        
        
def run_database_builder(input_path, db_path, drop_cols, comments_per_chunk=500000, chunk_size=100000):
    """
    args:
        input_path          - filepath to .json raw comments file
        db_path             - filepath to database file
        drop_cols           - columns to drop 
        comments_per_chunk  - number of comments to store in temporary .json files 
        chunk_size          - size of chunks for the pd.read_json() function
    """
    
    conn = create_engine('sqlite:///'+db_path)

    create_database(
        database=conn, 
        json_fp=input_path,
        comments_per_chunk=comments_per_chunk, 
        chunk_size=chunk_size,
        columns_to_drop=drop_cols
    )

In [3]:
DATA_ROOT = "/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/*/*.json"
DATA_DEST = "data/RC_2020_database.db"

In [7]:
data_files = sorted(glob.glob(DATA_ROOT))
print(data_files[102])
# n_comments = count_objects_in_file(filepath=data_files[0])
# print(n_comments)

/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-13.json


Specify some irrelevant columns to drop.

In [5]:
drop_cols = ["all_awardings", 
             "associated_award", 
             "author_flair_css_class", 
             "author_flair_richtext",
             "author_flair_background_color",
             "author_flair_text_color",
             "author_flair_type",
             "author_patreon_flair",
             "author_flair_template_id",
             "author_premium",
             "can_gild", 
             "collapsed",
             "collapsed_because_crowd_control",
             "collapsed_reason",
             "gildings",
             "permalink", 
             "subreddit_name_prefixed",
             "treatment_tags"]

Sanity check: Extract some comments from a file

In [6]:
comments = extract_subset(filepath=data_files[104], start=0, end=100)
df = pd.DataFrame(comments)
df = df.drop(columns=drop_cols)
df = drop_additional_columns(df=df)
df = df.astype(dtype_dict)
print(df.columns.values.tolist())

['author', 'author_created_utc', 'author_flair_text', 'author_fullname', 'awarders', 'body', 'can_mod_post', 'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded', 'id', 'is_submitter', 'link_id', 'locked', 'no_follow', 'parent_id', 'quarantined', 'removal_reason', 'retrieved_on', 'score', 'send_replies', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_type', 'top_awarded_type', 'total_awards_received']


In [8]:
df.head()

,author,author_created_utc,author_flair_text,author_fullname,awarders,body,can_mod_post,controversiality,created_utc,distinguished,edited,gilded,id,is_submitter,link_id,locked,no_follow,parent_id,quarantined,removal_reason,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,subreddit_type,top_awarded_type,total_awards_received
0,KlutzyDesign,NaN,None,t2_6esnza6x,[],"I’m summarizing a bible story for you, what do...",False,0,1592179200,None,False,0,fuumj80,False,t3_h8ouxr,False,True,t1_fuumg1t,False,None,1600998230,3,True,False,insanepeoplefacebook,t5_3acf2,public,None,0
1,santropez1972,1.421422e+09,None,t2_kr2tq,[],I know where I want it to go!,False,0,1592179200,None,False,0,fuumj81,False,t3_h93qni,False,False,t1_fuuirqk,False,None,1600998230,4,True,False,gilf,t5_2qvqa,public,None,0
2,kernals12,1.536085e+09,None,t2_23su9ek6,[],Yet another sign that we should've taken Canada.,False,0,1592179200,None,False,0,fuumj82,False,t3_h94skb,False,True,t3_h94skb,False,None,1600998230,-1,True,False,MapPorn,t5_2si92,public,None,0
3,nicernicer,NaN,None,t2_6982ih24,[],nice,False,0,1592179200,None,False,0,fuumj83,False,t3_h94ssz,False,True,t3_h94ssz,False,None,1600998230,2,True,False,lookatmydog,t5_2s6t5,public,None,0
4,amcg10,NaN,Bootlegger :Bootlegger:,t2_45mylopt,[],Okay fair enough you raise a good point still ...,False,0,1592179200,None,False,0,fuumj84,False,t3_h8wgtx,False,False,t1_fuum7h3,False,None,1600998230,4,True,False,apexlegends,t5_rgzzt,public,None,0


In [9]:
data_files[102:]

['/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-13.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-14.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-15.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-16.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-17.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-18.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-19.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-20.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-21.json',
 '/media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-22.json',
 '/media/c

In [ ]:
for idx, data_file in enumerate(data_files[102:]):
    t_start = time()
    run_database_builder(
        input_path=data_file, 
        db_path=DATA_DEST, 
        drop_cols=drop_cols,
    )
    t_end = time()
    t_iter = (t_end-t_start)/60
    print("\nFinished. Time: {:.1f} min".format(t_iter))


######## File: /media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-13.json
Contains 5834459 comments - dividing into 12 chunks
   File chunk 12: Extracting and writing comments 5500000 - 6000000Finished. Time: 28.0 min

######## File: /media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-14.json
Contains 5790092 comments - dividing into 12 chunks
   File chunk 12: Extracting and writing comments 5500000 - 6000000Finished. Time: 46.6 min

######## File: /media/cameron/Seagate Basic/datasets/reddit/archives_decompressed/jun_2020/RC_2020-06-15.json
Contains 6423523 comments - dividing into 13 chunks
   File chunk 3: Extracting and writing comments 1000000 - 1500000

In [ ]:
# t_start = time()
# run_database_builder(
#     input_path=data_files[101], 
#     db_path=DATA_DEST, 
#     drop_cols=drop_cols,
# )
# t_end = time()
# t_iter = (t_end-t_start)/60
# print("Finished. Time: {:.1f} min".format(t_iter))

In [ ]:
DB_NAME = DATA_DEST
conn = sqlite3.connect(DB_NAME)

In [ ]:
df = pd.read_sql_query("""SELECT * FROM reddit_comments LIMIT 100""", conn)
df = df.drop(columns=["index"])
df.head()